In [2]:
import tensorflow as tf 
import tensorflow_hub as hub
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten 
from tensorflow.keras.layers import Input
from tensorflow.keras.utils import to_categorical


2023-06-17 21:24:19.409992: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Directory where the images are located
dataset_dir = "/Users/alissa/Desktop/1_Practice/9.4/raw-img"

# List of animal classes
animal_classes = ['cat', 'dog', 'elephant', 'chicken', 'cow', 'sheep', 'spider', 'horse', 'squirrel']

# Lists to store images and labels
images = []
labels = []

# Loop over each animal class
for animal_class in animal_classes:
    # Create the path to the class directory
    class_dir = os.path.join(dataset_dir, animal_class)
    print(class_dir)
    
    # Loop over each image file in the class directory
    for image_file in os.listdir(class_dir):
        # Create the full image path
        image_path = os.path.join(class_dir, image_file)
        
        # Load the image and resize it to (224, 224)
        image = load_img(image_path, target_size=(224, 224))
        image = img_to_array(image)
        
        # Append the image and label to the respective lists
        images.append(image)
        labels.append(animal_class)

# Convert the lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Encode the labels with numeric values
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
print('done fitting')

# Split the data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)
print('done splitting')

# Normalize the pixel values to the range [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

# Convert the labels to one-hot encoded vectors
train_labels_one_hot = to_categorical(train_labels, num_classes=9)
test_labels_one_hot = to_categorical(test_labels, num_classes=9)

# Define the input shape for the model
input_shape = (224, 224, 3)
input_tensor = Input(shape=input_shape)

/Users/alissa/Desktop/1_Practice/9.4/raw-img/cat
/Users/alissa/Desktop/1_Practice/9.4/raw-img/dog
/Users/alissa/Desktop/1_Practice/9.4/raw-img/elephant
/Users/alissa/Desktop/1_Practice/9.4/raw-img/chicken
/Users/alissa/Desktop/1_Practice/9.4/raw-img/cow
/Users/alissa/Desktop/1_Practice/9.4/raw-img/sheep
/Users/alissa/Desktop/1_Practice/9.4/raw-img/spider
/Users/alissa/Desktop/1_Practice/9.4/raw-img/horse
/Users/alissa/Desktop/1_Practice/9.4/raw-img/squirrel
done fitting
done splitting


In [4]:
# Load the pre-trained InceptionV3 model without the top classification layer
base_model = InceptionV3(weights='imagenet', include_top=False)

# Pass the input tensor through the base model
x = base_model(input_tensor)

# Add additional layers on top of the base model
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((3, 3))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dense(128, activation='relu')(x)

# Add the final classification layer with 9 output units (one for each animal class)
predictions = Dense(9, activation='softmax')(x)

# Create the model with the input tensor as input and predictions as output
model = Model(inputs=input_tensor, outputs=predictions)

# Freeze the weights of the base model (do not update them during training)
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with Adam optimizer, categorical cross-entropy loss, and accuracy metric
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_images,
    train_labels_one_hot,
    batch_size=900,
    epochs=7,
    validation_data=(test_images, test_labels_one_hot)
)

Epoch 1/7
22/22 [==============================] - 1952s 87s/step - loss: 0.8427 - accuracy: 0.7649 - val_loss: 0.2194 - val_accuracy: 0.9435
Epoch 2/7
22/22 [==============================] - 1951s 89s/step - loss: 0.1658 - accuracy: 0.9536 - val_loss: 0.1622 - val_accuracy: 0.9578
Epoch 3/7
22/22 [==============================] - 2114s 97s/step - loss: 0.1182 - accuracy: 0.9635 - val_loss: 0.1548 - val_accuracy: 0.9585
Epoch 4/7
22/22 [==============================] - 2774s 128s/step - loss: 0.0891 - accuracy: 0.9734 - val_loss: 0.1464 - val_accuracy: 0.9605
Epoch 5/7
22/22 [==============================] - 2489s 114s/step - loss: 0.0701 - accuracy: 0.9796 - val_loss: 0.1306 - val_accuracy: 0.9630
Epoch 6/7
22/22 [==============================] - 2573s 119s/step - loss: 0.0541 - accuracy: 0.9853 - val_loss: 0.1336 - val_accuracy: 0.9645
Epoch 7/7
22/22 [==============================] - 1923s 88s/step - loss: 0.0393 - accuracy: 0.9897 - val_loss: 0.1351 - val_accuracy: 0.9636


In [5]:
# save the model
model.save("model_inception.h5") 